In [1]:
import numpy as np, pandas as pd, plotly.express as px, plotly.graph_objects as go
from pathlib import Path

from SJ_Haar_CNV import base_search
from SJ_Haar_CNV import decomposition
from tests import test_data

In [13]:
X = np.concatenate([np.repeat (5,130), np.repeat (7,190), np.repeat(5,100)]) + np.random.normal(0,1,420)
X

array([5.3667742 , 4.85782335, 4.8167225 , 4.8588647 , 4.81747854,
       4.67083285, 4.20384477, 3.87452254, 5.47303802, 4.93237007,
       3.26845437, 4.37722478, 4.30526237, 4.74830481, 4.73184891,
       6.45308417, 5.20865349, 5.17151852, 3.77436106, 4.79217456,
       5.59199342, 4.56652143, 3.94487235, 5.11859833, 6.27668762,
       5.42043712, 4.98164828, 5.12110727, 4.66050604, 4.10090686,
       4.72946241, 5.11044578, 5.44270859, 4.93969109, 3.82589718,
       4.76115526, 5.36725541, 4.29447841, 4.50317035, 4.49857485,
       5.26133603, 4.56226607, 4.94137279, 5.18749278, 4.6283875 ,
       5.40621194, 6.95259618, 6.40586724, 5.366835  , 7.95347727,
       6.72358992, 6.26302018, 5.79418392, 4.90537798, 5.9872164 ,
       5.0072751 , 4.33573204, 3.86790592, 5.74755897, 4.47741165,
       4.70966609, 5.32200286, 5.27481885, 4.49607858, 3.40258813,
       5.79765441, 4.87121407, 3.65951454, 3.87639778, 5.27435407,
       5.45799854, 3.87516241, 3.95166828, 4.47477625, 7.35939

In [14]:
basis = base_search.generate_haar_basis(X, p0=.95, length = 20, debug = False)
basis

[[0, 0, 0, (5.911882037722706, 420), (0, 0)],
 [0, 420, 0, (0.07251407362812771, 130), (-0.032756357397533555, 290)],
 [1, 130, 0, (0.25257668863597027, 13), (-0.030222851631654585, 117)],
 [1, 289, 131, (0.042601432284230495, 189), (-0.08094272134003795, 100)],
 [2, 116, 14, (0.30241195653113895, 9), (-0.02826279967580735, 107)],
 [2, 189, 131, (0.19548683580954593, 22), (-0.02692333666838059, 167)],
 [2, 99, 321, (0.21343747458109494, 17), (-0.04685212856658182, 82)],
 [3, 106, 24, (0.22243591216177602, 16), (-0.042015672297224374, 90)],
 [3, 22, 131, (0.5383819020581655, 2), (-0.08075728530872485, 20)],
 [3, 166, 154, (0.2914121942489626, 10), (-0.020548295748324272, 156)],
 [3, 81, 339, (0.3358643744858637, 7), (-0.03630966210657986, 74)],
 [4, 89, 41, (0.39440531887330776, 5), (-0.028171808490950533, 84)],
 [4, 155, 165, (0.323575114464717, 8), (-0.01981072129375818, 147)],
 [4, 73, 347, (0.43184081151100867, 4), (-0.03129281242833395, 69)],
 [5, 83, 47, (0.12297261940068255, 36),

In [15]:
res = decomposition.decompose (X, basis)

In [37]:
px.scatter (X)
px.line (decomposition.generate_function_from_wavelets (res.x, basis))
ind = np.array([0,  1,  3, 54, 52, 17])
allind = np.arange(len(basis))
px.line (decomposition.generate_function_from_wavelets (res.x[ind], [basis[i] for i in ind]), line_shape='spline')

In [31]:
res.x

array([  1.0152703 , -13.10880476,  -0.9298635 ,  15.75043318,
        -0.95025417,  -0.79707287,  -2.17234866,  -1.55022339,
         0.74895648,  -1.06840884,  -0.04927148,  -0.58409715,
         0.72147548,  -0.10796509,  -0.61694484,   0.53764003,
         1.41119374,   3.96637717,  -0.2659459 ,   0.62615991,
         0.52656309,  -0.74719434,   0.10763999,  -0.63541111,
        -0.3166988 ,  -0.15165424,   0.41757438,  -0.42266569,
        -1.00061717,  -0.67752545,  -0.1526709 ,  -2.15396721,
        -0.60388616,  -0.65713916,  -1.33859783,   1.54118912,
         2.54335155,  -0.98063915,  -0.78964053,   0.73338699,
        -1.04828891,  -0.49471763,   0.92688777,   1.50114965,
        -0.08162819,   0.21323583,   0.99816159,  -0.16818904,
        -0.98174   ,  -0.73734474,  -0.17363302,   0.42413256,
         2.85226807,   1.04013685,  -2.84940646,  -0.18850645,
         0.42022553])

In [23]:
v = np.abs (res.x)
px.scatter (x = np.sort(v), y = np.linspace (0,1,len(v)))

In [28]:
np.argsort(v)[-5:]

array([54, 52, 17,  1,  3], dtype=int64)

In [7]:
X.shape

(420,)